<a href="https://www.kaggle.com/code/yas1nth/bert-train?scriptVersionId=154925913" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

[](http://)

In [2]:
# load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_classes = 3
# model = BertModel.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes) # num_classes is the number of sentiment classes
model = model.cuda()

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import pandas as pd
import numpy as np

In [4]:
!wget https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSentences.csv
!wget https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSplit.csv

--2023-12-14 00:13:17--  https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSentences.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1290254 (1.2M) [text/plain]
Saving to: ‘datasetSentences.csv’

datasetSentences.cs 100%[===================>]   1.23M  --.-KB/s    in 0.04s   

2023-12-14 00:13:18 (34.9 MB/s) - ‘datasetSentences.csv’ saved [1290254/1290254]

--2023-12-14 00:13:19--  https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSplit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubuserc

In [5]:
df = pd.read_csv('datasetSentences.csv')
labels = pd.read_csv('datasetSplit.csv')

In [6]:
def inp_ids(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['input_ids']

def token_types(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['token_type_ids']

def attn_mask(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['attention_mask']

In [7]:
df['input_ids'] = df['sentence'].apply(inp_ids)
df['token_type_ids'] = df['sentence'].apply(token_types)
df['attention_mask'] = df['sentence'].apply(attn_mask)

In [8]:
def extract_embeddings(row):
    inputs = {
        'input_ids': row['input_ids'].cuda(),
        'token_type_ids': row['token_type_ids'].cuda(),
        'attention_mask': row['attention_mask'].cuda()
    }

    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        return embeddings.cpu().numpy()
    
def pad_tensor(tensor, max_length):
    padding_length = max_length - tensor.shape[1]
    if padding_length > 0:
        pad =(0, 0, 0, padding_length)
        tensor = torch.nn.functional.pad(tensor, pad, "constant", 0)
    return tensor

In [9]:
# df['embeddings'] = df[['input_ids','token_type_ids','attention_mask']].apply(extract_embeddings, axis=1)
# max_length = max([tensor.shape[1] for tensor in df.embeddings])
# X = [pad_tensor(torch.tensor(tensor), max_length) for tensor in df.embeddings]
# X = torch.vstack(X)
# X = [tensor.mean(dim=0) for tensor in X]
# X = torch.vstack(X)

In [10]:
y = labels['splitset_label'].values - 1
labels['splitset_label'].value_counts()

splitset_label
1    8544
2    2210
3    1101
Name: count, dtype: int64

In [11]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [12]:
batch_size = 1
learning_rate = 1e-5
num_epochs = 100

In [13]:
df['splitset_label'] = labels['splitset_label']

In [14]:
def pad_attn_mask(tensor, max_length):
    padding_length = max_length - tensor.shape[1]
    if padding_length > 0:
        pad =(0, padding_length)
        tensor = torch.nn.functional.pad(tensor, pad, "constant", 0)
    return tensor


In [15]:
# Pad each tensor in the DataFrame
max_length = max([tensor.shape[1] for tensor in df.input_ids])
x1 = [pad_attn_mask(torch.tensor(tensor), max_length) for tensor in df.input_ids]
x1 = torch.vstack(x1)

max_length = max([tensor.shape[1] for tensor in df.attention_mask])
x2 = [pad_attn_mask(torch.tensor(tensor), max_length) for tensor in df.attention_mask]
x2 = torch.vstack(x2)

/tmp/ipykernel_42/2433885622.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x1 = [pad_attn_mask(torch.tensor(tensor), max_length) for tensor in df.input_ids]
/tmp/ipykernel_42/2433885622.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x2 = [pad_attn_mask(torch.tensor(tensor), max_length) for tensor in df.attention_mask]


In [16]:
for item in (x1,x2):
    print(item[0].shape,item[1].shape)
    break

torch.Size([80]) torch.Size([80])


In [17]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Instantiate SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(x1, y)

xtrain, valtrain, ytrain, yval = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Instantiate SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(x2, y)


masktrain, maskval, _, _ = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
# dataset = TensorDataset(x1.cuda(), x2.cuda(), torch.tensor(y).cuda())
# train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [19]:
train_inputs = torch.tensor(xtrain)
validation_inputs = torch.tensor(valtrain)

train_labels = torch.tensor(ytrain)
validation_labels = torch.tensor(yval)

train_masks = torch.tensor(masktrain)
validation_masks = torch.tensor(maskval)

In [20]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#Creating the DataLoader which will help us to load data into the GPU/CPU
batch_size = 32*3

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, shuffle=True, batch_size=batch_size)


In [21]:
total_loss = 0
# Training loop
for epoch in range(num_epochs):
    model.train()
    for idx,batch in enumerate(train_dataloader):
        inputs, attention_mask, targets = batch[0].cuda(),batch[1].cuda(),batch[2].cuda()
#         inputs, attention_mask, targets = batch#[0].cuda(),batch[1].cuda(),batch[2].cuda()
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=targets)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    print("epoch", epoch,  "| loss", loss)

epoch 0 | loss tensor(0.8281, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 1 | loss tensor(0.7657, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 2 | loss tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 3 | loss tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 4 | loss tensor(0.5106, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 5 | loss tensor(0.6881, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 6 | loss tensor(0.5479, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 7 | loss tensor(0.3502, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 8 | loss tensor(0.2607, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 9 | loss tensor(0.1715, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 10 | loss tensor(0.1313, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 11 | loss tensor(0.0835, device='cuda:0', grad_fn=<NllLossBackward0>)
epoch 13 | loss tensor(0.0888, device='cuda:0', grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 

In [22]:
preds = []
act = []
with torch.no_grad():
    for idx,batch in enumerate(validation_dataloader):
        act.append(batch[2].numpy())
        inputs, attention_mask = batch[0].cuda(),batch[1].cuda()
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask)
        preds.append(outputs.logits.detach().cpu().numpy())

In [23]:
from torch.nn.functional import softmax

# Assuming `logits` is the output from your model and has shape [batch_size, num_classes]
tmp_logits = torch.tensor(np.vstack(preds))
probabilities = softmax(tmp_logits, dim=1)  # Apply softmax to convert logits to probabilities
predicted_labels = torch.argmax(probabilities, dim=1)

In [24]:
act_labels = []
for arr in act:
    act_labels.extend(arr.tolist())

In [25]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(predicted_labels, act_labels)
report = classification_report(predicted_labels, act_labels)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.5396918275794812
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.72      0.73      1756
           1       0.49      0.40      0.44      2095
           2       0.39      0.51      0.44      1276

    accuracy                           0.54      5127
   macro avg       0.54      0.55      0.54      5127
weighted avg       0.55      0.54      0.54      5127

